<a href="https://colab.research.google.com/github/eyaler/avatars4all/blob/master/ganivut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo for paper "Liquid Warping GAN with Attention: A Unified Framework for Human Image Synthesis (Impersonator++)"

### Made just a little bit more accessible by Eyal Gruss (https://eyalgruss.com, eyalgruss@gmail.com)

##### Original project: https://www.impersonator.org/work/impersonator-plus-plus.html

##### Original notebook: <https://colab.research.google.com/drive/1bwUnj-9NnJA2EMr7eWO4I45UuBtKudg_>

#### **Stuff I made**:
##### Avatars4all repository: https://github.com/eyaler/avatars4all
##### Notebook for live webcam in the browser: https://colab.research.google.com/github/eyaler/avatars4all/blob/master/fomm_live.ipynb
##### Notebook for talking head model: https://colab.research.google.com/github/eyaler/avatars4all/blob/master/fomm_bibi.ipynb
##### Notebook for full body models (FOMM): https://colab.research.google.com/github/eyaler/avatars4all/blob/master/fomm_fufu.ipynb
##### Notebook for full body models (impersonator): https://colab.research.google.com/github/eyaler/avatars4all/blob/master/ganozli.ipynb
##### Notebook for full body models (impersonator++): https://colab.research.google.com/github/eyaler/avatars4all/blob/master/ganivut.ipynb
##### Notebook for Wav2Lip audio based lip syncing: https://colab.research.google.com/github/eyaler/avatars4all/blob/master/melaflefon.ipynb
##### List of more generative tools: https://j.mp/generativetools

In [ ]:
#@title Setup
%cd /content
!git clone --depth 1 https://github.com/eyaler/iPERCore

!apt-get install ffmpeg
import os
os.environ["CUDA_HOME"] = "/usr/local/cuda-10.1"

%cd /content/iPERCore
!python setup.py develop
%cd /content

!wget -nc -O /content/iPERCore/assets/checkpoints.zip "http://101.32.75.151:12345/iper_plus_plus_latest_checkpoints.zip"
!wget -nc -O /content/iPERCore/assets/checkpoints.zip "https://eyalgruss.com/fomm/iper_plus_plus_latest_checkpoints.zip"
!unzip -u /content/iPERCore/assets/checkpoints.zip -d /content/iPERCore/assets/

#!wget -nc -O /content/iPERCore/assets/samples.zip  "http://101.32.75.151:12345/iper_plus_plus_latest_samples.zip"
#!unzip -u /content/iPERCore/assets/samples.zip -d /content/iPERCore/assets

!pip install -U youtube-dl
!pip install imageio==2.9.0
!pip install imageio-ffmpeg==0.4.5


In [ ]:
#@title Get the Driver video, Avatar and Background images from the web
#@markdown 1. You can change the URLs to your **own** stuff!
#@markdown 2. Alternatively, you can upload **local** files in the next cells
 
video_url = 'https://www.youtube.com/watch?v=NdEPmitJvaA' #@param {type:"string"}
image_url = 'https://i.pinimg.com/564x/74/9e/96/749e96293399ebe1b8c99d89be522383.jpg' #@param {type:"string"}
background_url = 'https://c.pxhere.com/photos/a9/9a/forest_tree_trees_foliage_leaves_leaf_nature_needles-1005931.jpg!d' #@param {type:"string"}

if video_url:
  !rm -f /content/video.mp4
  !youtube-dl -f 'bestvideo[ext=mp4][vcodec!*=av01][height<=720]+bestaudio[ext=m4a]/mp4[height<=720][vcodec!*=av01]/mp4[vcodec!*=av01]/mp4' '$video_url' --merge-output-format mp4 -o /content/video
  !mv /content/video.mp4 /content/video 
 
if image_url:
  !wget '$image_url' -O /content/image

if background_url:
  !wget '$background_url' -O /content/background

In [ ]:
#@title Optionally upload local Driver video { run: "auto" }
manually_upload_video = False #@param {type:"boolean"}
if manually_upload_video:
  from google.colab import files
  import shutil

  %cd /content/sample_data
  try:
    uploaded = files.upload()
  except Exception as e:
    %cd /content
    raise e

  for fn in uploaded:
    shutil.move('/content/sample_data/'+fn, '/content/video')
    break
  %cd /content

In [ ]:
#@title Optionally upload local Avatar image { run: "auto" }
manually_upload_image = False #@param {type:"boolean"}
if manually_upload_image:
  from google.colab import files
  import shutil
 
  %cd /content/sample_data
  try:
    uploaded = files.upload()
  except Exception as e:
    %cd /content
    raise e

  for fn in uploaded:
    shutil.move('/content/sample_data/'+fn, '/content/image')
    break
  %cd /content

In [ ]:
#@title Optionally upload local Background image { run: "auto" }
manually_upload_background = False #@param {type:"boolean"}
if manually_upload_background:
  from google.colab import files
  import shutil
 
  %cd /content/sample_data
  try:
    uploaded = files.upload()
  except Exception as e:
    %cd /content
    raise e

  for fn in uploaded:
    shutil.move('/content/sample_data/'+fn, '/content/background')
    break
  %cd /content

In [ ]:
#@title Optionally shorten Driver video
start_seconds = 0 #@param {type:"number"}
duration_seconds =  60#@param {type:"number"}
start_seconds = max(start_seconds,0)
duration_seconds = max(duration_seconds,0)

if duration_seconds: 
  !mv /content/video /content/full_video
  !ffmpeg -ss $start_seconds -t $duration_seconds -i /content/full_video -f mp4 /content/video -y

In [ ]:
#@title Prepare assets
#@markdown If you ran out of RAM this means that the video is too large. You can shorten it above.
 
#center_video_to_body = True #@param {type:"boolean"}
#crop_video_to_body = True #@param {type:"boolean"}
#video_crop_expansion_factor = 1.05 #@param {type:"number"}
center_image_to_body = True #@param {type:"boolean"}
crop_image_to_body = False #@param {type:"boolean"}
image_crop_expansion_factor = 1.05 #@param {type:"number"}
#video_crop_expansion_factor = max(video_crop_expansion_factor, 1)
image_crop_expansion_factor = max(image_crop_expansion_factor, 1)
keep_aspect_background = True #@param {type:"boolean"}
image_size = 512

import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML, clear_output
import cv2
import shutil
import os
import warnings
warnings.filterwarnings("ignore")
 
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
 
def fix_dims(im):
    if im.ndim == 2:
        im = np.tile(im[..., None], [1, 1, 3])
    return im[...,:3]
 
def get_crop(im, center_body=True, crop_body=True, expansion_factor=1, rects=None):
    im = fix_dims(im)
    if (center_body or crop_body) and rects is None:
        rects, _ = hog.detectMultiScale(im, winStride=(4, 4),padding=(8,8), scale=expansion_factor)
    if (center_body or crop_body) and rects is not None and len(rects):
        x0,y0,w,h = sorted(rects, key=lambda x: x[2]*x[3])[-1]
        if crop_body:
            x0 += w//2-h//2
            x1 = x0+h
            y1 = y0+h
        else:
            img_h,img_w = im.shape[:2]
            x0 += (w-img_h)//2
            x1 = x0+img_h
            y0 = 0
            y1 = img_h
    else:
        h,w = im.shape[:2]
        x0 = (w-h)//2
        x1 = (w+h)//2
        y0 = 0
        y1 = h
    return int(x0),int(x1),int(y0),int(y1)
 
def pad_crop_resize(im, x0=None, x1=None, y0=None, y1=None, new_h=image_size, new_w=image_size):
    im = fix_dims(im)
    h,w = im.shape[:2]
    if x0 is None:
      x0 = 0
    if x1 is None:
      x1 = w
    if y0 is None:
      y0 = 0
    if y1 is None:
      y1 = h
    if x0<0 or x1>w or y0<0 or y1>h:
        im = np.pad(im, pad_width=[(max(-y0,0),max(y1-h,0)),(max(-x0,0),max(x1-w,0)),(0,0)], mode='edge')
    return resize(im[max(y0,0):y1-min(y0,0),max(x0,0):x1-min(x0,0)], (new_h, new_w))

def crop_resize(im, size, crop=False):
  if im.shape[:2] == size:
    return im
  if size[0]<im.shape[0] or size[1]<im.shape[1]:
    interp = cv2.INTER_AREA
  else:
    interp = cv2.INTER_CUBIC
  if not crop:
    return np.clip(cv2.resize(im, size[::-1], interpolation=interp),0,1)
  ratio = max(size[0]/im.shape[0], size[1]/im.shape[1])
  im = np.clip(cv2.resize(im, (int(np.ceil(im.shape[1]*ratio)), int(np.ceil(im.shape[0]*ratio))), interpolation=interp),0,1)
  return im[(im.shape[0]-size[0])//2:(im.shape[0]-size[0])//2+size[0], (im.shape[1]-size[1])//2:(im.shape[1]-size[1])//2+size[1]]
 
source_image = imageio.imread('/content/image')
source_image = pad_crop_resize(source_image, *get_crop(source_image, center_body=center_image_to_body, crop_body=crop_image_to_body, expansion_factor=image_crop_expansion_factor))
imageio.imwrite('/content/crop.png', (source_image*255).astype(np.uint8))

if os.path.exists('/content/background'):
  bg_image = imageio.imread('/content/background')
  bg_image = crop_resize(bg_image/255, source_image.shape[:2], crop=keep_aspect_background)
  imageio.imwrite('/content/bg_crop.png', (bg_image*255).astype(np.uint8))

#shutil.rmtree('/content/images', ignore_errors=True)
#os.makedirs('/content/images')
with imageio.get_reader('/content/video', format='mp4') as reader:
  fps = reader.get_meta_data()['fps']
''' 
  driving_video = []
  rects = None
  try:
      for i,im in enumerate(reader):
          if not crop_video_to_body:
              break
          rects, _ = hog.detectMultiScale(im, winStride=(4, 4),padding=(8,8), scale=video_crop_expansion_factor)
          if rects is not None and len(rects):
              break
      x0,x1,y0,y1 = get_crop(im, center_body=center_video_to_body, crop_body=crop_video_to_body, expansion_factor=video_crop_expansion_factor, rects=rects)
      reader.set_image_index(0)
      for j,im in enumerate(reader):
          vid_frame = pad_crop_resize(im,x0,x1,y0,y1)
          #driving_video.append(vid_frame)
          imageio.imwrite(os.path.join('/content/images','%05d.jpg'%j), (vid_frame*255).astype(np.uint8))
  except RuntimeError:
      pass
 
def vid_display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
 
    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])
 
    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
 
clear_output()
if rects is not None and len(rects):
    print('first found body in frame %d'%i)
print('number of frames: %d'%j)
HTML(vid_display(source_image, driving_video).to_html5_video())
'''
pass

In [ ]:
#@title Animate
#@markdown 1. The 'pose_fc' is the smooth factor of the temporal poses. The smaller of this value, the smoother of the temporal poses.
#@markdown 2. The 'cam_fc' is the smooth factor of the temporal cameras (locations in the image space). The smaller of this value, the smoother of the locations in sequences.

pose_fc = 300 #@param {type:'integer'}
cam_fc = 100 #@param {type:'integer'}
background = 'replace' #@param ['none', 'inpaint', 'replace']

%cd /content/iPERCore

import os.path as osp
import platform
import argparse
import time
import sys
import subprocess
from IPython.display import HTML
from base64 import b64encode

# the gpu ids
gpu_ids = "0"

# the default number of source images, it will be updated if the actual number of sources <= num_source
num_source = 1

# the assets directory. This is very important, please download it from `one_drive_url` firstly.
assets_dir = "/content/iPERCore/assets"

# the output directory.
output_dir = "./results"
shutil.rmtree(output_dir, ignore_errors=True)

# symlink from the actual assets directory to this current directory
work_asserts_dir = os.path.join("./assets")
if not os.path.exists(work_asserts_dir):
    os.symlink(osp.abspath(assets_dir), osp.abspath(work_asserts_dir),
               target_is_directory=(platform.system() == "Windows"))

cfg_path = osp.join(work_asserts_dir, "configs", "deploy.toml")

# This is a specific model name, and it will be used if you do not change it. This is the case of `trump`
model_id = "mymodel"

# the source input information, here \" is escape character of double duote "
src_path = "\"path?=/content/crop.png,name?=mymodel"
if background=='replace' and os.path.exists('/content/bg_crop.png'):
  src_path += ',bg_path?=/content/bg_crop.png'
src_path += '"'

!cp /content/video /content/video.mp4
ref_path = "\"path?=/content/video.mp4,"  \
             "name?=myoutput," \
             "pose_fc?=%d,"\
             "cam_fc?=%d,"\
             "fps?=%f\""%(pose_fc,cam_fc,fps)
options = ''
if background=='inpaint':
  options += ' --use_inpaintor'
!python -m iPERCore.services.run_imitator --gpu_ids $gpu_ids --num_source $num_source --image_size $image_size --output_dir $output_dir --model_id $model_id --cfg_path $cfg_path --src_path $src_path --ref_path $ref_path $options

clear_output()
mp4 = open("./results/primitives/mymodel/synthesis/imitations/mymodel-myoutput.mp4", "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls autoplay loop>
      <source src="{data_url}" type="video/mp4">
</video>""")


In [ ]:
#@title Download
#@markdown 1. If it fails try running this cell again.
#@markdown 2. Alternatively, you can manually download "output.mp4", "combined.mp4" from the folder on the left (click "Refresh" if missing).

print() #see https://github.com/googlecolab/colabtools/issues/468
from google.colab import files
files.download('./results/primitives/mymodel/synthesis/imitations/mymodel-myoutput.mp4') #fails for Firefox private window